In [ ]:
from pathlib import Path
import pydicom
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
labels = pd.read_csv("stage_2_train_labels.csv")
labels = labels.drop_duplicates("patientId")

ROOT_PATH = Path("stage_2_train_images/")
SAVE_PATH = Path("processed/")

In [ ]:
sums, sums_squared = 0, 0

for c, patient_id in enumerate(tqdm(labels.patientId)):
        patient_id = labels.patientId.iloc[c]
        dcm_path = ROOT_PATH/patient_id
        dcm_path = dcm_path.with_suffix(".dcm")
        dcm = pydicom.read_file(dcm_path).pixel_array / 255
        
        dcm_array = cv2.resize(dcm, (224, 224)).astype(np.float16)
        label = labels.Target.iloc[c]
        
        train_or_val = "train" if c < 24000 else "val"
        current_save_path = SAVE_PATH/train_or_val/str(label)
        current_save_path.mkdir(parents=True, exist_ok=True)
        np.save(current_save_path/patient_id, dcm_array)
        
        normalizer = 224*224
        if train_or_val == 'train':
            sums += np.sum(dcm_array) / normalizer
            sums_squared += (dcm_array ** 2).sum() / normalizer
            